# UU TaX at SemEval 2022: *TASK3: PRETENS - SUBTASK2*




Cloning the git repository, to get data for Task3:subtask-2


# Imports

In [ ]:
!pip install tensorflow_text 

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from matplotlib import pyplot

In [ ]:
!pip3 install tensorflow_text>=2.0.0rc0


In [ ]:
##### Data Reader -- English/French/Italian
!git clone https://github.com/shammur/SemEval2022Task3.git
%cd ./SemEval2022Task3/
#data/train/train_subtask-2/
!ls

Setting up the environment for the task

In [ ]:
import os, sys, time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn.svm import SVR
from scipy import stats
from collections import defaultdict
import numpy as np
import pandas as pd
import statistics

from baseline.utils import get_train_dev, compute_metrics_task2, process_eval_jsons_task2

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)



# Download USE


In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text


model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3") #large



#  Regression

## Models

### USE

In [ ]:
def use(train,test, classifier='svm'):
  eval_jsons=[]

  def getUse(df):
    return model([df['Sentence']])[0].numpy()


  #Get Features 
  train['Sentence_emb']=train.apply(getUse,axis=1)
  train_x_feat=train['Sentence_emb']


  test['Sentence_emb']=test.apply(getUse,axis=1)
  test_x_feat=test['Sentence_emb']

  #Get text and lables
  train_texts=(train['Sentence'])
  y_train=np.array(train['Score'])
  
  test_texts=(test['Sentence'])
  test_true_y=np.array(test['Score'])   

  
  if classifier=='lr':
        regresson =LinearRegression().fit(list(train_x_feat), y_train)
  elif classifier=='knr':
        regresson =KNeighborsRegressor().fit(list(train_x_feat), y_train)
  elif classifier=='dt':
        regresson =DecisionTreeRegressor().fit(list(train_x_feat), y_train)
  else:
      regresson =SVR(C=1.0, epsilon=0.2).fit(list(train_x_feat), y_train)
    
  #Testing:
  test_pred_y = regresson.predict(list(test_x_feat))
    
  eval_jsons.append(compute_metrics_task2(test_pred_y,test_true_y))
  return eval_jsons,test_pred_y

### TF_IDF and and send classifier name: svm, knr, lr, dt (For baseline testing)

In [ ]:
def tfidf(train,test, classifier='svm', ngram=3):
  eval_jsons=[]

 

  #Get text and lables
  train_texts=(train['Sentence'])
  y_train=np.array(train['Score'])
  
  test_texts=(test['Sentence'])
  test_true_y=np.array(test['Score'])   

  #Creating ngram features
  tfidf_vectorizer = TfidfVectorizer(encoding='utf-8', lowercase=True, ngram_range=(1, ngram), norm='l1',
                                       use_idf=True, max_df=0.95, min_df=3, max_features=1000) 

  train_x_feat = tfidf_vectorizer.fit_transform(train_texts)


  #Training Linear Model
  if classifier=='lr':
        regresson =LinearRegression().fit(train_x_feat, y_train)
  elif classifier=='knr':
        regresson =KNeighborsRegressor().fit(train_x_feat, y_train)
  elif classifier=='dt':
        regresson =DecisionTreeRegressor().fit(train_x_feat, y_train)
  else:
        regresson =SVR(C=1.0, epsilon=0.2).fit(train_x_feat, y_train)
    
  #Testing:
  test_x_feat = tfidf_vectorizer.transform(test_texts)
  test_pred_y = regresson.predict(test_x_feat)
    
  eval_jsons.append(compute_metrics_task2(test_pred_y,test_true_y))
  return eval_jsons,test_pred_y

# English

## Train

In [ ]:
data_url = 'https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/train/train_subtask-2/en/'
folds = [pd.read_csv(os.path.join(data_url, 'En-Subtask2-fold_{}.tsv'.format(i)), sep='\t') \
         for i in range(2)] #reading all folds from GitHub

df_en_train = pd.concat(folds[0:2]) 
df_en_train

,ID,Sentence,Score
0,34,"I like cats, but not sparrows.",4.77
1,256,"I like caviar, an interesting type of veal.",1.83
2,118,"I like turkey, but not oysters.",5.42
3,363,"I like crabs, and chicken too.",5.75
4,307,"I like oaks, and trees too.",4.00
...,...,...,...
257,248,"I like oysters, an interesting type of turkey.",1.83
258,277,"I like ducks, and birds too.",4.83
259,255,"I like seafood, an interesting type of caviar.",1.67
260,375,"I like oysters, and seafood too.",4.45


## Read  test data

In [ ]:
#dummy score
import random
scores=[random.uniform(1.0,7.0)]*1009 
len(scores)

1009

In [ ]:
import base64
import requests
import pandas as pd
df_en_test = pd.read_csv('https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/test/official_test_sets/subtask-2/En-Subtask2-test.tsv', sep='\t')
df_en_test=df_en_test.set_index('ID')
df_en_test['Score']=scores
df_en_test 

,Sentence,Score
ID,,
en_0,I like governors more than farmers.,4.17382
en_1,"Unlike parrots, cats are often mentioned in th...",4.17382
en_2,"Unlike pines, bushes are often mentioned in th...",4.17382
en_3,"I like oaks, and bushes in general.",4.17382
en_4,"I like beef, and oysters in general.",4.17382
...,...,...
en_1004,"I like hamsters, and in particular sparrows.",4.17382
en_1005,"Unlike forests, concrete is often mentioned in...",4.17382
en_1006,"I like trees, and in particular oaks.",4.17382


## Call best model and Save best output English

In [ ]:
classifiers=['svm','knr','dt','lr']
max_rho=(float('-inf'),"")

In [ ]:
for classifier in classifiers:
  results,scores=use(df_en_train,df_en_test,classifier)

In [ ]:
for ngram in range(2,5):  
  for classifier in classifiers:
    results,scores=tfidf(df_en_train,df_en_test,classifier,ngram)
    print('English results on model--TF-IDF--->',classifier,' --> n grams:  ',ngram)
    print(results)

call final model

In [ ]:
en_results,en_scores=use(df_en_train,df_en_test,'svm')
en_results

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


[{'eval_labels': array([4.17382041, 4.17382041, 4.17382041, ..., 4.17382041, 4.17382041,
         4.17382041]),
  'eval_pred': array([4.83009547, 4.38265232, 4.16041686, ..., 3.05867929, 2.36724716,
         3.96661342]),
  'mse': 0.397325149857144,
  'p-value': nan,
  'rho': nan,
  'rmse': 0.6303373302106928}]

In [ ]:
df_en_test['Score']=en_scores
del df_en_test['Sentence_emb']
df_en_test

,Sentence,Score
ID,,
en_0,I like governors more than farmers.,4.830095
en_1,"Unlike parrots, cats are often mentioned in th...",4.382652
en_2,"Unlike pines, bushes are often mentioned in th...",4.160417
en_3,"I like oaks, and bushes in general.",4.200822
en_4,"I like beef, and oysters in general.",4.418871
...,...,...
en_1004,"I like hamsters, and in particular sparrows.",3.610608
en_1005,"Unlike forests, concrete is often mentioned in...",3.990365
en_1006,"I like trees, and in particular oaks.",3.058679


In [ ]:
from google.colab import files
df_en_test.to_csv('answer_english_with_sent.csv') 
files.download('answer_english_with_sent.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# French

## Train

In [ ]:
data_url = 'https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/train/train_subtask-2/fr/'
folds = [pd.read_csv(os.path.join(data_url, 'Fr-Subtask2-fold_{}.tsv'.format(i)), sep='\t') \
         for i in range(2)] #reading all folds from GitHub

df_fr_train = pd.concat(folds[0:2]) 
df_fr_train

,ID,Sentence,Score
0,110,"J'adore le boeuf, mais pas les crabes.",5.08
1,230,"J'adore le boeuf, un type intéressant de saumon.",1.00
2,36,"J'aime les oiseaux, mais pas les moineaux.",5.92
3,128,"J'adore la dinde, mais pas le caviar.",5.92
4,114,"J'adore les produits de la mer, mais pas les h...",6.08
...,...,...,...
257,224,"J'adore le veau, un type intéressant de saumon.",1.08
258,249,"J'adore la dinde, un type intéressant de huîtres.",1.17
259,143,"J'aime les oiseaux, un type intéressant de chien.",1.42
260,88,"J'adore le poulet, mais pas le saumon.",6.08


In [ ]:
import base64
import requests
import pandas as pd
df_fr_test = pd.read_csv('https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/test/official_test_sets/subtask-2/Fr-Subtask2-test.tsv', sep='\t')
df_fr_test=df_fr_test.set_index('ID')
scores=[random.uniform(1.0,7.0)]*1009 

df_fr_test['Score']=scores
df_fr_test 

,Sentence,Score
ID,,
fr_0,"J'aime les buissons, et les sapins en général.",2.367089
fr_1,"J'aime les produits de la mer, et le caviar en...",2.367089
fr_2,"J'apprécie les conducteurs, et en particulier ...",2.367089
fr_3,"J'apprécie les politiciens, et en particulier ...",2.367089
fr_4,"J'aime les huîtres, et le veau en général.",2.367089
...,...,...
fr_1004,"J'apprécie les maires, et en particulier les c...",2.367089
fr_1005,"J'aime les arbres, et en particulier les buiss...",2.367089
fr_1006,"Contrairement à le saumon, le poulet est souve...",2.367089


## Call best model and Save best output French

In [ ]:
classifiers=['svm','knr','dt','lr']
max_rho=(float('-inf'),"")

In [ ]:
for classifier in classifiers:
  results,scores=use(df_fr_train,df_fr_test,classifier)
  print('French results on model--USE--->',classifier)
  print(results)

In [ ]:
for ngram in range(2,5):  
  for classifier in classifiers:
    results,scores=tfidf(df_fr_train,df_fr_test,classifier,ngram)
    print('French results on model--TF-IDF--->',classifier,' --> n grams:  ',ngram)
    print(results)

In [ ]:
fr_results,fr_scores=use(df_fr_train,df_fr_test,'svm')
fr_results

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


[{'eval_labels': array([2.36708881, 2.36708881, 2.36708881, ..., 2.36708881, 2.36708881,
         2.36708881]),
  'eval_pred': array([5.36274696, 4.28460405, 3.53859671, ..., 3.8638618 , 3.96695587,
         4.16541158]),
  'mse': 3.0314114671480854,
  'p-value': nan,
  'rho': nan,
  'rmse': 1.7410949046930455}]

In [ ]:
df_fr_test['Score']=fr_scores
del df_fr_test['Sentence_emb']
df_fr_test

,Sentence,Score
ID,,
fr_0,"J'aime les buissons, et les sapins en général.",5.362747
fr_1,"J'aime les produits de la mer, et le caviar en...",4.284604
fr_2,"J'apprécie les conducteurs, et en particulier ...",3.538597
fr_3,"J'apprécie les politiciens, et en particulier ...",3.515586
fr_4,"J'aime les huîtres, et le veau en général.",5.091562
...,...,...
fr_1004,"J'apprécie les maires, et en particulier les c...",3.681623
fr_1005,"J'aime les arbres, et en particulier les buiss...",4.806597
fr_1006,"Contrairement à le saumon, le poulet est souve...",3.863862


In [ ]:
df_fr_test.to_csv('answer_fr_with_sent.csv') 
files.download('answer_fr_with_sent.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Italian

## Train

In [ ]:
data_url = 'https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/train/train_subtask-2/it/'
folds = [pd.read_csv(os.path.join(data_url, 'It-Subtask2-fold_{}.tsv'.format(i)), sep='\t') \
         for i in range(2)] #reading all folds from GitHub

df_it_train = pd.concat(folds[0:2]) 
df_it_train

,ID,Sentence,Score
0,73,"Adoro gli abeti, ma non i cespugli.",4.33
1,106,"Adoro il vitello, ma non i granchi.",3.33
2,203,"Adoro gli animali, un tipo interessante di abete.",1.25
3,466,Amo gli abeti più dei cespugli.,5.18
4,137,"Amo i criceti, un tipo interessante di pappaga...",1.36
...,...,...,...
257,370,"Amo il tacchino, ed anche i granchi.",6.00
258,453,Amo gli alberi più delle betulle.,2.22
259,288,"Amo gli uccelli, ed anche i merli.",3.55
260,386,"Amo i prodotti del mare, ed anche il caviale.",5.08


In [ ]:

import base64
import requests
import pandas as pd
df_it_test = pd.read_csv('https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/test/official_test_sets/subtask-2/It-Subtask2-test.tsv', sep='\t')
df_it_test=df_it_test.set_index('ID')
scores=[random.uniform(1.0,7.0)]*1009 

df_it_test['Score']=scores
df_it_test 

,Sentence,Score
ID,,
it_0,"Amo gli arbusti, e le querce in generale.",6.126631
it_1,"Amo le montagne, e in particolare i centri com...",6.126631
it_2,Amo le foreste più delle case.,6.126631
it_3,"Amo le foreste, ed anche i centri commerciali.",6.126631
it_4,"Amo i merli, e i gatti in generale.",6.126631
...,...,...
it_1004,Apprezzo i senatori più degli agricoltori.,6.126631
it_1005,"A differenza dei prodotti del mare, il vitello...",6.126631
it_1006,Apprezzo i politici più degli agricoltori.,6.126631


## Call best model and Save best output Italian

In [ ]:
classifiers=['svm','knr','dt','lr']
max_rho=(float('-inf'),"")

In [ ]:
for classifier in classifiers:
  results,scores=use(df_it_train,df_it_test,classifier)
  print('Italian results on model--USE--->',classifier)
  print(results)

In [ ]:
for ngram in range(2,5):  
  for classifier in classifiers:
    results,scores=tfidf(df_it_train,df_it_test,classifier,ngram)
    print('Italian results on model--TF-IDF--->',classifier,' --> n grams:  ',ngram)
    print(results)

In [ ]:
it_results,it_scores=use(df_it_train,df_it_test,'svm')
it_results

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


[{'eval_labels': array([6.12663091, 6.12663091, 6.12663091, ..., 6.12663091, 6.12663091,
         6.12663091]),
  'eval_pred': array([4.42420179, 3.35511671, 4.08052385, ..., 4.12250227, 4.23677037,
         3.6671752 ]),
  'mse': 7.174213327024914,
  'p-value': nan,
  'rho': nan,
  'rmse': 2.6784722001590597}]

In [ ]:
df_it_test['Score']=it_scores
del df_it_test['Sentence_emb']
df_it_test

,Sentence,Score
ID,,
it_0,"Amo gli arbusti, e le querce in generale.",4.424202
it_1,"Amo le montagne, e in particolare i centri com...",3.355117
it_2,Amo le foreste più delle case.,4.080524
it_3,"Amo le foreste, ed anche i centri commerciali.",3.906703
it_4,"Amo i merli, e i gatti in generale.",4.504795
...,...,...
it_1004,Apprezzo i senatori più degli agricoltori.,4.102093
it_1005,"A differenza dei prodotti del mare, il vitello...",2.997645
it_1006,Apprezzo i politici più degli agricoltori.,4.122502


In [ ]:
df_it_test.to_csv('answer_it_with_sent.csv') 
files.download('answer_it_with_sent.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Concat All best_outputs for 3 langs

In [ ]:
df_en_test

,Sentence,Score
ID,,
en_0,I like governors more than farmers.,4.830095
en_1,"Unlike parrots, cats are often mentioned in th...",4.382652
en_2,"Unlike pines, bushes are often mentioned in th...",4.160417
en_3,"I like oaks, and bushes in general.",4.200822
en_4,"I like beef, and oysters in general.",4.418871
...,...,...
en_1004,"I like hamsters, and in particular sparrows.",3.610608
en_1005,"Unlike forests, concrete is often mentioned in...",3.990365
en_1006,"I like trees, and in particular oaks.",3.058679


In [ ]:
del df_en_test['Sentence']
df_en_test

,Score
ID,
en_0,4.830095
en_1,4.382652
en_2,4.160417
en_3,4.200822
en_4,4.418871
...,...
en_1004,3.610608
en_1005,3.990365
en_1006,3.058679


In [ ]:
df_en_test.to_csv('answer_en.csv') 
files.download('answer_en.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_fr_test

,Sentence,Score
ID,,
fr_0,"J'aime les buissons, et les sapins en général.",5.362747
fr_1,"J'aime les produits de la mer, et le caviar en...",4.284604
fr_2,"J'apprécie les conducteurs, et en particulier ...",3.538597
fr_3,"J'apprécie les politiciens, et en particulier ...",3.515586
fr_4,"J'aime les huîtres, et le veau en général.",5.091562
...,...,...
fr_1004,"J'apprécie les maires, et en particulier les c...",3.681623
fr_1005,"J'aime les arbres, et en particulier les buiss...",4.806597
fr_1006,"Contrairement à le saumon, le poulet est souve...",3.863862


In [ ]:
del df_fr_test['Sentence']
df_fr_test

,Score
ID,
fr_0,5.362747
fr_1,4.284604
fr_2,3.538597
fr_3,3.515586
fr_4,5.091562
...,...
fr_1004,3.681623
fr_1005,4.806597
fr_1006,3.863862


In [ ]:
df_fr_test.to_csv('answer_fr.csv') 
files.download('answer_fr.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_it_test

,Sentence,Score
ID,,
it_0,"Amo gli arbusti, e le querce in generale.",4.424202
it_1,"Amo le montagne, e in particolare i centri com...",3.355117
it_2,Amo le foreste più delle case.,4.080524
it_3,"Amo le foreste, ed anche i centri commerciali.",3.906703
it_4,"Amo i merli, e i gatti in generale.",4.504795
...,...,...
it_1004,Apprezzo i senatori più degli agricoltori.,4.102093
it_1005,"A differenza dei prodotti del mare, il vitello...",2.997645
it_1006,Apprezzo i politici più degli agricoltori.,4.122502


In [ ]:
del df_it_test['Sentence']
df_it_test

,Score
ID,
it_0,4.424202
it_1,3.355117
it_2,4.080524
it_3,3.906703
it_4,4.504795
...,...
it_1004,4.102093
it_1005,2.997645
it_1006,4.122502


In [ ]:
df_it_test.to_csv('answer_it.csv') 
files.download('answer_it.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>